# Process margin texts of birth certificates of Suriname 1828-1921

In [1]:
import os
import pandas as pd
import re
import sys
sys.path.append(os.getcwd() + '/..')
from scripts import ner_analysis

2023-04-25 13:59:12.513995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 13:59:12.605113: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 13:59:12.607738: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 13:59:12.607751: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

## 1. Read data

In [2]:
data_file = "../../data/kantmeldingen.csv"

In [3]:
data = pd.read_csv(data_file)

/tmp/ipykernel_15946/254111080.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_file)


## 2. Check data

In [ ]:
len(data)

In [ ]:
for key in data:
    print(key, end=" ")

In [ ]:
birth_date_list = list(data["birth_date"])
birth_year_list = []
for birth_date in birth_date_list:
    try:
        birth_year = birth_date[6:10]
        if len(birth_year) == 4:
            birth_year_list.append(int(birth_year))
    except:
        pass
print(sorted(birth_year_list)[:10], sorted(birth_year_list)[-10:])

In [ ]:
def find_rows(column, value):
    return data[data[column].apply(lambda x: bool(re.search(value, x, re.IGNORECASE)))]

In [ ]:
find_rows("birth_date", "1988")["birth_date"]

In [ ]:
data["note_type"].value_counts(dropna=False)

In [ ]:
data["note_type_other"].value_counts(dropna=False)[:15]

In [ ]:
data["note_txt"][:20]

In [ ]:
note_txt_summary = []
for note_txt in data["note_txt"]:
    try:
        string = note_txt.strip().lower()
        string = re.sub("\d\d\-\d\d-\d\d\d\d", "00-00-0000", string)
        string = re.sub("door .* als", "door x als", string)
        string = re.sub("overl\. ", "overleden ", string)
        string = re.sub("o\. *p\. ", "overleden te paramaribo ", string)
        string = re.sub("par'*bo", "paramaribo", string)
        string = re.sub("paramb?o", "paramaribo", string)
        string = re.sub("paramaribo[.,]", "paramaribo", string)
        string = re.sub("paramaribo op ", "paramaribo ", string)
        string = re.sub("^o\. ", "overleden ", string)
        string = re.sub("^o ", "overleden ", string)
        string = re.sub(" p ", " paramaribo ", string)
        string = re.sub("overl ", "overleden ", string)
        string = re.sub("overleden par", "overleden te par", string)
        string = re.sub("0000 is .* in tegenstaande", "0000 is z in tegenstaande", string)
        #string = re.sub("bij akte verleden voor den ambtenaar van den burgerlijken stand te paramaribo *den 00-00-0000,? is het in tegenstaande akte genoemd kind door x als (haar|hun|zijn) natuurlijk kind erkend\.?", "", string)
        note_txt_summary.append(string)
        #note_txt_summary.append(re.sub("\d\d\-\d\d-\d\d\d\d", "00-00-0000", note_txt))
    except:
        note_txt_summary.append("NaN")
data["note_txt_summary"] = note_txt_summary     

In [ ]:
value_counts = data["note_txt_summary"].value_counts(dropna=False)
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['Value', 'Count']
value_counts_df = value_counts_df[['Count', 'Value']]
value_counts_df[:10]

In [ ]:
find_rows("note_txt_summary", "akte verleden")[:10]["note_txt_summary"]

In [ ]:
find_rows("note_txt_summary", "akte verleden")[:10]["note_txt_summary"][2902]

## 3. Find entities

In [4]:
ner_analysis = ner_analysis.NerAnalysis()

In [5]:
text = data["note_txt"][9]
entity_tokens = ner_analysis.process(data["note_txt"][9])
ner_analysis.render_text(text, entity_tokens)